In [ ]:
import pymysql

# DB 접속 정보
host = 'localhost'
user = 'python'
password = 'python'
db_name = 'python_db'
table_name = 'test'

def save_to_DB(url, feature_names, importances, result):
    conn = pymysql.connect(host=host, user=user, password=password, db=db_name, charset='utf8mb4')
    try:
        with conn.cursor() as cursor:
            # 1. 테이블 생성 SQL 구성 ("url"과 "result" 열 추가)
            columns_sql = (
                '`url` TEXT UNIQUE,\n' +
                ',\n'.join([f"`{name}` FLOAT" for name in feature_names]) + ',\n' +
                '`result` FLOAT'
            )
            create_table_sql = f"""
            CREATE TABLE IF NOT EXISTS `{table_name}` (
                id INT AUTO_INCREMENT PRIMARY KEY,
                {columns_sql}
            )
            """
            cursor.execute(create_table_sql)

            # 2. URL 존재 여부 확인
            cursor.execute(f"SELECT COUNT(*) FROM `{table_name}` WHERE url = %s", (url,))
            exists = cursor.fetchone()[0] > 0

            if exists:
                # 3-1. UPDATE SQL 구성
                set_clause = ', '.join([f"`{name}` = %s" for name in feature_names] + ['`result` = %s'])
                update_sql = f"UPDATE `{table_name}` SET {set_clause} WHERE url = %s"
                update_data = list(importances) + [result, url]
                cursor.execute(update_sql, update_data)
                print("🔄 기존 URL 정보 업데이트 완료")
            else:
                # 3-2. INSERT SQL 구성
                columns = '`url`, ' + ', '.join([f"`{name}`" for name in feature_names]) + ', `result`'
                placeholders = '%s, ' + ', '.join(['%s'] * len(feature_names)) + ', %s'
                insert_sql = f"INSERT INTO `{table_name}` ({columns}) VALUES ({placeholders})"
                data = [url] + list(importances) + [result]
                cursor.execute(insert_sql, data)
                print("✅ 새 URL 정보 삽입 완료")

        conn.commit()
    finally:
         conn.close()
